# Needleman-Wunsch Algorithm

This algorithm, developed by Saul B. Needleman and Christian D. Wunsch, is utilized for global sequence alignment of amino acids and nucleotides.

In [8]:
import numpy as np
import pandas as pd

In [9]:
def read_fasta(filename):
    with open(filename, 'r') as f:
        sequence = ""
        for line in f:
            line = line.strip()
            if line.startswith(">"): #ignore first line with label
                pass
            else:
                sequence += line
    return sequence

In [107]:
class NeedlemanWunsch:
    '''
    Contains the Needleman-Wunsch score & matrix for two sequences.

    Important Attributes:
    score (int) : the score for the matching algorithm
    alignment (tuple (str, str)) : the alignments for sequence 1 and 2

    Important Methods:
    display_matrix() : displays the scored matrix as a DataFrame
                       where light blue is the traceback method.
    '''
    def __init__(self, seq1, seq2, match = 1, mismatch = -1, gap = -1):
        self.seq1 = seq1
        self.seq2 = seq2
        self.match = match
        self.mismatch = mismatch
        self.gap = gap
    
        self.rows = len(seq1) + 1
        self.columns = len(seq2) + 1

        self.fill_matrix()

        self.score = self.find_score()
        self.alignment = self.traceback()

    def init_matrix(self):
        self.matrix = np.zeros((self.rows, self.columns), dtype = int)
        self.matrix[0, :] = np.arange(0, (self.columns) * self.gap, self.gap)
        self.matrix[:, 0] = np.arange(0, (self.rows) * self.gap, self.gap)

    def fill_matrix(self):

        self.init_matrix()

        for i in range(1, self.rows):                   #iterate through each cell of the matrix
            for j in range(1, self.columns):

                if self.seq1[i - 1] == self.seq2[j - 1]: #if the same, it's a match
                    match_score = self.match
                else:                                    #otherwise, penalize as a mismatch
                    match_score = self.mismatch
            
                up = self.matrix[i - 1, j] + self.gap
                left = self.matrix[i, j - 1] + self.gap
                diagonal = self.matrix[i - 1, j - 1] + match_score

                self.matrix[i, j] = max(diagonal, up, left)     #save the best score

    def find_score(self):
        #score is the bottom right cell of the matrix
        return self.matrix[self.rows - 1 , self.columns - 1]
    
    def traceback(self):
        i = self.rows  - 1
        j = self.columns - 1

        align1, align2 = "", ""
        traceback_coords = []

        while i > 0 and j > 0:
            traceback_coords.append((i, j))
            current = self.matrix[i, j]
            x1 = self.seq1[i - 1]
            x2 = self.seq2[j - 1]


            if x1 == x2: #determine score
                match_score = self.match
            else:
                match_score = self.mismatch

            #diagonal
            if current == self.matrix[i - 1, j - 1] + match_score:
                align1 = x1 + align1
                align2 = x2 + align2
                i -= 1
                j -= 1

            #up
            elif current == self.matrix[i - 1, j] + self.gap:
                align1 = x1 + align1
                align2 = "-" + align2
                i -= 1

            #left
            else:
                align1 = "-" + align1
                align2 = x2 + align2
                j -= 1

        #fill rest with gaps if one side already aligned
        while i > 0:
            traceback_coords.append((i, j))
            align1 = self.seq1[i - 1] + align1
            align2 = "-" + align2
            i -= 1

        while j > 0:
            traceback_coords.append((i, j))
            align1 = "-" + align1
            align2 = self.seq2[j - 1] + align2
            j -= 1

        self.traceback_coords = traceback_coords
        return align1, align2

    def display_matrix(self):
        '''
        Returns the scoring matrix with the traceback labeled in red
        '''
        df = pd.DataFrame(self.matrix) 

        styled_df = df.style.apply( lambda row: [
            "background-color: crimson" 
            if (row.name, col_idx) in self.traceback_coords else ""
            for col_idx in range(df.shape[1]) ], axis=1 )
        

        return styled_df


Short Sequence Example:

In [108]:
example = NeedlemanWunsch("AGGTA", "ACGT")
print(f"Score: {example.score}\n")
print(f"Alignment:\n{example.alignment[0]}\n{example.alignment[1]}")

example.display_matrix()

Score: 1

Alignment:
AGGTA
ACGT-


,0,1,2,3,4
0,0,-1,-2,-3,-4
1,-1,1,0,-1,-2
2,-2,0,0,1,0
3,-3,-1,-1,1,0
4,-4,-2,-2,0,2
5,-5,-3,-3,-1,1


## Real World Examples

In [109]:
human_hbb = read_fasta("data/HBB_HUMAN.fasta")
mouse_hbb = read_fasta("data/HBB1_MOUSE.fasta")
pigeon_hbb = read_fasta("data/HBB_COLLI.fasta")

Human Hemoglobin VS Mouse Hemoglobin

In [110]:
humanVmouse = NeedlemanWunsch(human_hbb, mouse_hbb)
print(f"Score: {humanVmouse.score}\n")
print(f"Alignment:\n{humanVmouse.alignment[0]}\n{humanVmouse.alignment[1]}")

Score: 89

Alignment:
MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH
MVHLTDAEKAAVSCLWGKVNSDEVGGEALGRLLVVYPWTQRYFDSFGDLSSASAIMGNAKVKAHGKKVITAFNDGLNHLDSLKGTFASLSELHCDKLHVDPENFRLLGNMIVIVLGHHLGKDFTPAAQAAFQKVVAGVATALAHKYH


Human Hemoglobin VS Rock Dove Hemoglobin

In [111]:
humanVpigeon = NeedlemanWunsch(human_hbb, pigeon_hbb)
print(f"Score: {humanVpigeon.score}\n")
print(f"Alignment:\n{humanVpigeon.alignment[0]}\n{humanVpigeon.alignment[1]}")

Score: 56

Alignment:
MVHLTPEEKSAVTALWGKVNV-DEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH
-VHWSAEEKQLITSIWGKVNVAD-CGAEALARLLIVYPWTQRFFSSFGNLSSATAISGNPNVKAHGKKVLTSFGDAVKNLDNIKGTFAQLSELHCDKLHVDPENFRLLGDILVIILAAHFGKDFTPECQAAWQKLVRVVAHALARKYH
